In [1]:
!pip install --upgrade pip
!pip install opencv-python    
!pip install pillow

  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
     -------------------------------------- 35.4/35.4 MB 242.6 kB/s eta 0:00:00


# First we reduce dimensions because the data are very big

In [ ]:
# write to anaconda prompt: jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
# jupyter notebook --notebook-dir=D:/
from PIL import Image
import os
path='chest_xray_lower_dim'
old_path='chest_xray/'
isExist = os.path.exists(path)
if not isExist:
    os.makedirs(path)
import cv2
import glob
files=glob.glob('chest_xray/*.jpeg')
dataset=[]
labels=[]
import PIL
from PIL import Image
import cv2
for file in files:
    img=Image.open(file)
    img=img.convert(mode='L')#too much information without reason x-rays are BlackWhite
    img=img.resize((400,400))#we try to stop curse of dimensionality
    img.save('chest_xray_lower_dim/'+file[len(old_path):])
print('Done')

### The data has lower dimension but it takes too long to process them so we reduce to 100x100

In [1]:
# write to anaconda prompt: jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
from PIL import Image
import os
path='chest_xray_lower_dim2'
old_path='chest_xray_lower_dim/'
isExist = os.path.exists(path)
if not isExist:
    os.makedirs(path)
import cv2
import glob
files=glob.glob('chest_xray_lower_dim/*.jpeg')
dataset=[]
labels=[]
import PIL
from PIL import Image
import cv2
for file in files:
    img=Image.open(file)
    img=img.convert(mode='L')#too much information without reason x-rays are BlackWhite
    img=img.resize((100,100))#we try to stop curse of dimensionality
    img.save('chest_xray_lower_dim2/'+file[len(old_path):])
print('Done')

Done


### Afterwards we create the dataset with the labels and we will try to make a prediction system about these

In [2]:
#before work
import cv2
import glob
files=glob.glob('chest_xray_lower_dim2/*.jpeg')
dataset=[]
labels=[]
for file in files:
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    dataset.append(img)
    #write your code
    if "virus" in file:
        labels.append("0")
    elif "bacteria" in file:
        labels.append("1")
    else:
        labels.append("2")
        
from numpy import asarray
dataset=asarray(dataset).astype('float32')
dataset=dataset/255.0

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np

In [2]:
!pip install tensorflow

     -------------------------------------- 438.0/438.0 MB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 42.6/42.6 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 5.8/5.8 MB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 14.2/14.2 MB 3.2 MB/s eta 0:00:00
     -------------------------------------- 126.7/126.7 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 462.5/462.5 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 65.5/65.5 kB 3.5 MB/s eta 0:00:00
     -------------------------------------- 781.3/781.3 kB 3.1 MB/s eta 0:00:00
     -------------------

#### First try

In [14]:
X_train,X_test,Y_train,Y_test=train_test_split(
dataset, labels, test_size=0.2, random_state=42)
X_train=X_train.reshape(-1,100,100,1)
X_test=X_test.reshape(-1,100,100,1)
Y_train = keras.utils.to_categorical(Y_train,3)
Y_test = keras.utils.to_categorical(Y_test,3)
Inp=layers.Input(shape=(100,100,1))
hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(Inp)
hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
hidden=layers.AveragePooling2D(pool_size=(2,2),strides=(2,2))(hidden)
hidden=layers.Flatten()(hidden)
for i in range(5):
    hidden=layers.Dense(30,activation="relu",kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/76.4, maxval=1/76.4, seed=None))(hidden)
output=layers.Dense(3,activation="softmax")(hidden)
model=keras.Model(Inp,output)
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=tf.keras.metrics.CategoricalAccuracy())
model.fit(X_train,Y_train,epochs=int(len(X_train)/200),batch_size=200,verbose=0)
model.summary()
score = model.evaluate(X_test, Y_test,verbose=2)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 100, 100, 1)]     0         
                                                                 
 conv2d_18 (Conv2D)          (None, 98, 98, 10)        100       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 49, 49, 10)       0         
 2D)                                                             
                                                                 
 conv2d_19 (Conv2D)          (None, 47, 47, 10)        910       
                                                                 
 average_pooling2d_9 (Averag  (None, 23, 23, 10)       0         
 ePooling2D)                                                     
                                                                 
 flatten_9 (Flatten)         (None, 5290)              0   

#### let's test it with 10 layers

In [3]:
#με αυτό το score για 2η φάση α
X_train,X_test,Y_train,Y_test=train_test_split(
dataset, labels, test_size=0.2, random_state=42)
X_train=X_train.reshape(-1,100,100,1)
X_test=X_test.reshape(-1,100,100,1)
Y_train = keras.utils.to_categorical(Y_train,3)
Y_test = keras.utils.to_categorical(Y_test,3)
Inp=layers.Input(shape=(100,100,1))
hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(Inp)
hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
hidden=layers.AveragePooling2D(pool_size=(2,2),strides=(2,2))(hidden)
hidden=layers.Flatten()(hidden)
for i in range(10):
    hidden=layers.Dense(30,activation="relu",kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/76.4, maxval=1/76.4, seed=None))(hidden)
output=layers.Dense(3,activation="softmax")(hidden)
model=keras.Model(Inp,output)
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=tf.keras.metrics.CategoricalAccuracy())
model.fit(X_train,Y_train,epochs=int(len(X_train)/100),batch_size=200,verbose=0)
model.summary()
score = model.evaluate(X_test, Y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 10)        100       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 10)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 10)        910       
                                                                 
 average_pooling2d (AverageP  (None, 23, 23, 10)       0         
 ooling2D)                                                       
                                                                 
 flatten (Flatten)           (None, 5290)              0     

#### That was worst,lets reduce the neurons to 3 

In [3]:
#με αυτό το score για 2η φάση α
X_train,X_test,Y_train,Y_test=train_test_split(
dataset, labels, test_size=0.2, random_state=42)
X_train=X_train.reshape(-1,100,100,1)
X_test=X_test.reshape(-1,100,100,1)
Y_train = keras.utils.to_categorical(Y_train,3)
Y_test = keras.utils.to_categorical(Y_test,3)
Inp=layers.Input(shape=(100,100,1))
hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(Inp)
hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
hidden=layers.AveragePooling2D(pool_size=(2,2),strides=(2,2))(hidden)
hidden=layers.Flatten()(hidden)
for i in range(3):
    hidden=layers.Dense(30,activation="relu",kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/76.4, maxval=1/76.4, seed=None))(hidden)
output=layers.Dense(3,activation="softmax")(hidden)
model=keras.Model(Inp,output)
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=tf.keras.metrics.CategoricalAccuracy())
model.fit(X_train,Y_train,epochs=int(len(X_train)/100),batch_size=200,verbose=0)
model.summary()
score = model.evaluate(X_test, Y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 10)        100       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 10)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 10)        910       
                                                                 
 average_pooling2d (AverageP  (None, 23, 23, 10)       0         
 ooling2D)                                                       
                                                                 
 flatten (Flatten)           (None, 5290)              0     

### Three layers are good let's test the number of neurons

In [4]:
#3 layers is good lets see number of neurons
X_train,X_test,Y_train,Y_test=train_test_split(
dataset, labels, test_size=0.2, random_state=42)
X_train=X_train.reshape(-1,100,100,1)
X_test=X_test.reshape(-1,100,100,1)
Y_train = keras.utils.to_categorical(Y_train,3)
Y_test = keras.utils.to_categorical(Y_test,3)
Inp=layers.Input(shape=(100,100,1))
hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(Inp)
hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
hidden=layers.AveragePooling2D(pool_size=(2,2),strides=(2,2))(hidden)
hidden=layers.Flatten()(hidden)
for i in range(3):
    hidden=layers.Dense(60,activation="relu",kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/76.4, maxval=1/76.4, seed=None))(hidden)
output=layers.Dense(3,activation="softmax")(hidden)
model=keras.Model(Inp,output)
model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=tf.keras.metrics.CategoricalAccuracy())
model.fit(X_train,Y_train,epochs=int(len(X_train)/100),batch_size=200,verbose=0)
model.summary()
score = model.evaluate(X_test, Y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 98, 98, 10)        100       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 49, 49, 10)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 47, 47, 10)        910       
                                                                 
 average_pooling2d_1 (Averag  (None, 23, 23, 10)       0         
 ePooling2D)                                                     
                                                                 
 flatten_1 (Flatten)         (None, 5290)              0   

#### let's make function to avoid repeated code

In [5]:
#3 layers is good lets see number of neurons
def train_neurons(neurons):
    X_train,X_test,Y_train,Y_test=train_test_split(
    dataset, labels, test_size=0.2, random_state=42)
    X_train=X_train.reshape(-1,100,100,1)
    X_test=X_test.reshape(-1,100,100,1)
    Y_train = keras.utils.to_categorical(Y_train,3)
    Y_test = keras.utils.to_categorical(Y_test,3)
    Inp=layers.Input(shape=(100,100,1))
    hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(Inp)
    hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
    hidden=layers.AveragePooling2D(pool_size=(2,2),strides=(2,2))(hidden)
    hidden=layers.Flatten()(hidden)
    for i in range(3):
        hidden=layers.Dense(neurons,activation="relu",kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/76.4, maxval=1/76.4, seed=None))(hidden)
    output=layers.Dense(3,activation="softmax")(hidden)
    model=keras.Model(Inp,output)
    model.compile(loss="categorical_crossentropy",optimizer='adam',metrics=tf.keras.metrics.CategoricalAccuracy())
    model.fit(X_train,Y_train,epochs=int(len(X_train)/100),batch_size=200,verbose=0)
    model.summary()
    score = model.evaluate(X_test, Y_test)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])


In [6]:
train_neurons(40)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 98, 98, 10)        100       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 49, 49, 10)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 47, 47, 10)        910       
                                                                 
 average_pooling2d_2 (Averag  (None, 23, 23, 10)       0         
 ePooling2D)                                                     
                                                                 
 flatten_2 (Flatten)         (None, 5290)              0   

In [7]:
train_neurons(20)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d_6 (Conv2D)           (None, 98, 98, 10)        100       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 49, 49, 10)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 47, 47, 10)        910       
                                                                 
 average_pooling2d_3 (Averag  (None, 23, 23, 10)       0         
 ePooling2D)                                                     
                                                                 
 flatten_3 (Flatten)         (None, 5290)              0   

In [8]:
train_neurons(35)

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d_8 (Conv2D)           (None, 98, 98, 10)        100       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 49, 49, 10)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 47, 47, 10)        910       
                                                                 
 average_pooling2d_4 (Averag  (None, 23, 23, 10)       0         
 ePooling2D)                                                     
                                                                 
 flatten_4 (Flatten)         (None, 5290)              0   

##### We quess that 30 neurons are better,let's test the learning rate

In [10]:
#best model has 30 neurons
X_train,X_test,Y_train,Y_test=train_test_split(
dataset, labels, test_size=0.2, random_state=42)
X_train=X_train.reshape(-1,100,100,1)
X_test=X_test.reshape(-1,100,100,1)
Y_train = keras.utils.to_categorical(Y_train,3)
Y_test = keras.utils.to_categorical(Y_test,3)
Inp=layers.Input(shape=(100,100,1))
hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(Inp)
hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
hidden=layers.AveragePooling2D(pool_size=(2,2),strides=(2,2))(hidden)
hidden=layers.Flatten()(hidden)
for i in range(3):
    hidden=layers.Dense(30,activation="relu",kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/76.4, maxval=1/76.4, seed=None))(hidden)
output=layers.Dense(3,activation="softmax")(hidden)
model=keras.Model(Inp,output)
model.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),metrics=tf.keras.metrics.CategoricalAccuracy())
model.fit(X_train,Y_train,epochs=int(len(X_train)/100),batch_size=200,verbose=0)
model.summary()
score = model.evaluate(X_test, Y_test)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d_12 (Conv2D)          (None, 98, 98, 10)        100       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 49, 49, 10)       0         
 2D)                                                             
                                                                 
 conv2d_13 (Conv2D)          (None, 47, 47, 10)        910       
                                                                 
 average_pooling2d_6 (Averag  (None, 23, 23, 10)       0         
 ePooling2D)                                                     
                                                                 
 flatten_6 (Flatten)         (None, 5290)              0   

#### learning rate is high.Let's reduce it rather than increase it

In [3]:
def train_rate_neurons(neurons,learn):
    X_train,X_test,Y_train,Y_test=train_test_split(
    dataset, labels, test_size=0.2, random_state=42)
    X_train=X_train.reshape(-1,100,100,1)
    X_test=X_test.reshape(-1,100,100,1)
    Y_train = keras.utils.to_categorical(Y_train,3)
    Y_test = keras.utils.to_categorical(Y_test,3)
    Inp=layers.Input(shape=(100,100,1))
    hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(Inp)
    hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
    hidden=layers.Conv2D(10,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
    hidden=layers.AveragePooling2D(pool_size=(2,2),strides=(2,2))(hidden)
    hidden=layers.Flatten()(hidden)
    for i in range(3):
        hidden=layers.Dense(neurons,activation="relu",kernel_initializer=tf.keras.initializers.RandomUniform(minval=-1/76.4, maxval=1/76.4, seed=None))(hidden)
    output=layers.Dense(3,activation="softmax")(hidden)
    model=keras.Model(Inp,output)
    model.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=learn),metrics=tf.keras.metrics.CategoricalAccuracy())
    model.fit(X_train,Y_train,epochs=int(len(X_train)/100),batch_size=200,verbose=0)
    model.summary()
    score = model.evaluate(X_test, Y_test)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])


In [5]:
train_rate_neurons(30,0.0001)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 98, 98, 10)        100       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 10)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 10)        910       
                                                                 
 average_pooling2d (AverageP  (None, 23, 23, 10)       0         
 ooling2D)                                                       
                                                                 
 flatten (Flatten)           (None, 5290)              0     

#### We think that best learning rate for now is the default